In [2]:
import pandas as pd
import numpy as np
import preprocessing

In [3]:
#loading data
df_female = pd.read_csv('test_data2/df_female.csv')
df_age = pd.read_csv('test_data2/df_age.csv')
df_SFI = pd.read_csv('test_data2/df_SFI.csv')
df_uti = pd.read_csv('test_data2/df_uti.csv')
df_admissions = pd.read_csv('test_data2/df_admissions.csv')
df_FIM_total = pd.read_csv('test_data2/df_FIM_total.csv')
df_acute_days = pd.read_csv('test_data2/df_acute_days.csv')
df_brain_injury = pd.read_csv('test_data2/df_brain_injury_mod.csv')

#Removing extra column
df_female = df_female.loc[:, ~df_female.columns.str.contains('^Unnamed')]
df_age = df_age.loc[:, ~df_age.columns.str.contains('^Unnamed')]
df_SFI = df_SFI.loc[:, ~df_SFI.columns.str.contains('^Unnamed')]
df_uti = df_uti.loc[:, ~df_uti.columns.str.contains('^Unnamed')]
df_admissions = df_admissions.loc[:, ~df_admissions.columns.str.contains('^Unnamed')]
df_FIM_total = df_FIM_total.loc[:, ~df_FIM_total.columns.str.contains('^Unnamed')]
df_acute_days = df_acute_days.loc[:, ~df_acute_days.columns.str.contains('^Unnamed')]
df_brain_injury = df_brain_injury.loc[:, ~df_brain_injury.columns.str.contains('^Unnamed')]


#rename FIM_total to value (column-name needs to be value for timeseriesflattener)
df_FIM_total = df_FIM_total.rename(columns= {'FIM_total': 'value'})



In [4]:
#Creating uti df with negative cases as well
merged = pd.merge(df_age, df_uti, on=['ID'], how='left')

merged = merged[['ID','value']]

#turning UVI-neg cases from nan to 0
where_are_NaNs = np.isnan(merged)
merged[where_are_NaNs] = 0

#checking
print(merged)

#Changing from floats to int
merged['value'] = merged['value'].astype('int')

merged = pd.merge(merged, df_uti, on=['ID', 'value'], how='left')

print(merged)

           ID  value
0        1436    1.0
1        2056    0.0
2        2492    1.0
3        3084    0.0
4        3087    0.0
...       ...    ...
1859  8169844    0.0
1860  8267752    1.0
1861  8769379    0.0
1862  8814557    0.0
1863  8869412    0.0

[1864 rows x 2 columns]
           ID  value        date
0        1436      1  2020-08-10
1        2056      0         NaN
2        2492      1  2019-02-04
3        3084      0         NaN
4        3087      0         NaN
...       ...    ...         ...
1859  8169844      0         NaN
1860  8267752      1  2020-12-09
1861  8769379      0         NaN
1862  8814557      0         NaN
1863  8869412      0         NaN

[1864 rows x 3 columns]


In [5]:
#just a test of how many patients have how X notes

few = 100
i=0
k=0

for ID in df_SFI['ID'].unique():
    k += 1
    df_SFI_sub = df_SFI[df_SFI['ID'] == ID]
    if len(df_SFI_sub) < few:
        i += 1
        #print(len(df_tfidfvect_train_sub))

print(f"Out of: {k} patients, {i} have fewer than {few} notes. That is {int(i/k*100)}% ")
i

Out of: 1864 patients, 1373 have fewer than 100 notes. That is 73% 


1373

In [6]:
df_age = df_age.rename(columns= {'age': 'value'})
df_female = df_female.rename(columns= {'sex': 'value'})
df_acute_days = df_acute_days.rename(columns= {' acute_days': 'value'})


df_acute_days

,ID,value
0,3377919,NaN
1,4361311,9.0
2,4458004,39.0
3,4384550,50.0
4,5008712,12.0
...,...,...
1859,4954644,33.0
1860,6076316,53.0
1861,6326651,82.0
1862,6621963,28.0


In [7]:
#Important to call predictors static, dynamic or text, otherwise they will be skipped because the function will not know how to handle them!
#Make a disclaimer message if there is such a df in the predictor dict
predictor_dict = {'df_SFI_text':df_SFI, 'df_age_static':df_age, 'df_female_static':df_female, 'df_acute_days_static':df_acute_days, 'df_FIM_total_dynamic':df_FIM_total }#, 'df_brain_injury_static':df_brain_injury}  # 

In [8]:
predictor_dict_train, predictor_dict_test, y_train_df, y_test_df, df_admissions_train, df_admissions_test = preprocessing.data_split(df_outcome = merged, predictor_dict = predictor_dict, df_admissions = df_admissions, test_size = 0.3)

Number of observations in trainingset:362
Percentage of positive class in trainingset: 27.76073619631902
Number of observations in testset:156
Percentage of positive class in testset: 27.857142857142858


In [9]:
from tfidf import tf_idf

df_tfidfvect_train, df_tfidfvect_test = tf_idf(predictor_dict_train["df_SFI_text"], predictor_dict_test["df_SFI_text"], min_df = 5, max_features=200)

e:\Projekter\RHN\linapd\tfidf_hyperparamsearch.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  X_train_SFI['note'] = X_train_SFI['note'].str.replace('\d+', '')
e:\Projekter\RHN\linapd\tfidf_hyperparamsearch.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  X_test_SFI['note'] = X_test_SFI['note'].str.replace('\d+', '')


Tokens ['adspurgt' 'aff' 'afføring' 'aftalt' 'aften' 'aftenvagten' 'av' 'bad'
 'beder' 'behandling' 'behov' 'besøg' 'ble' 'bleen' 'bleer' 'bleskift'
 'blevet' 'blod' 'blære' 'blæren' 'blærescannet' 'brugt' 'bukseble'
 'bukser' 'bundskift' 'bundskiftet' 'bækken' 'bækkenstol' 'ca' 'dag' 'dd'
 'diurese' 'dv' 'efterfølgende' 'ej' 'evt' 'faldet' 'fik' 'fint'
 'forbindelse' 'forsøgt' 'fortsat' 'fortæller' 'fungerer' 'fået' 'får'
 'føler' 'gang' 'gange' 'gerne' 'giver' 'givet' 'godt' 'grundet' 'gul'
 'gå' 'gået' 'går' 'haft' 'held' 'hele' 'helt' 'hjælp' 'hjælpes' 'holde'
 'hvil' 'hvilket' 'ifm' 'ildelugtende' 'inden' 'kad' 'kalder' 'kateter'
 'kath' 'kl' 'klar' 'kolbe' 'kolben' 'komme' 'kommet' 'konc'
 'koncentreret' 'kort' 'lade' 'lang' 'let' 'ligger' 'lys' 'læge' 'løbet'
 'ml' 'morgen' 'muligt' 'mængde' 'mærke' 'mærker' 'mørk' 'nat' 'natten'
 'nedre' 'nitrit' 'nåede' 'når' 'obs' 'par' 'pga' 'pose' 'posen' 'positiv'
 'prøve' 'pt' 'påsat' 'relevant' 'resultat' 'ringer' 'samt' 'sat' 'scan'
 's

In [10]:
# from sentence_embeddings import sentence_embeddings

# df_sentence_train = sentence_embeddings(df_free_text = predictor_dict_train["df_SFI_text"], transformer_model = 'encoder-large-v1')

# df_sentence_test = sentence_embeddings(df_free_text = predictor_dict_test["df_SFI_text"], transformer_model = 'encoder-large-v1')

In [11]:
# To test a smaller subset

ID_list_train  = []
[ID_list_train.append(i) for i in df_tfidfvect_train['ID'].unique()]

small_ID_list_train = ID_list_train[:100]

df_tfidfvect_train_small = df_tfidfvect_train[df_tfidfvect_train['ID'].isin(small_ID_list_train)]
df_admissions_train_small = df_admissions_train[df_admissions_train['ID'].isin(small_ID_list_train)]
y_train_df_small = y_train_df[y_train_df['ID'].isin(small_ID_list_train)]

ID_list_test = []
[ID_list_test.append(i) for i in df_tfidfvect_test['ID'].unique()]

small_ID_list_test = ID_list_test[:40]

df_tfidfvect_test_small = df_tfidfvect_test[df_tfidfvect_test['ID'].isin(small_ID_list_test)]
df_admissions_test_small = df_admissions_test[df_admissions_test['ID'].isin(small_ID_list_test)]
y_test_df_small = y_test_df[y_test_df['ID'].isin(small_ID_list_test)]

## same, but for sentence (the other df's are the same)
#df_sentence_train_small = df_sentence_train[df_sentence_train['ID'].isin(small_ID_list)]

In [12]:
#TESTING

from create_tensors import max_notes, tensors
import torch

# '''REMEMBER: possible to manually set look_back and look_ahead
# Look_back must be set to the same in both max_notes and tensors to create correct padding.
# (default look_back = 4, look_ahead = 3)'''


# #hvad hvis der er en i testsættet der har flere notes?? skal vel tjekke begge og tage den med flest??
# maximum_notes_train = max_notes(df_tfidfvect_train_small, df_admissions_train_small)
# print(maximum_notes_train)

# maximum_notes_test = max_notes(df_tfidfvect_test_small, df_admissions_test_small)
# print(maximum_notes_test)

# maximum_notes = max(maximum_notes_train, maximum_notes_test)
# print(maximum_notes)
maximum_notes = 32

#train
X_tfidfvect_train_tensor, y_tfidfvect_train_tensor, X_structured_train_tensor_tfidf = tensors(df_tfidfvect_train_small, y_train_df_small, df_admissions_train_small, maximum_notes, predictor_dict_train)
torch.save(X_tfidfvect_train_tensor, 'X_tfidfvect_train_tensor_4_days_test.pt')
torch.save(y_tfidfvect_train_tensor, 'y_tfidfvect_train_tensor_4_days_test.pt')
# torch.save(X_static_train_tensor, 'X_static_train_tensor_4_days_test.pt')
# torch.save(X_dynamic_train_tensor, 'X_dynamic_train_tensor_4_days_test.pt')
torch.save(X_structured_train_tensor_tfidf, 'X_structured_train_tensor_tfidf_4_days_subset.pt')

#test
X_tfidfvect_test_tensor, y_tfidfvect_test_tensor, X_structured_test_tensor_tfidf = tensors(df_tfidfvect_test_small, y_test_df_small, df_admissions_test_small, maximum_notes, predictor_dict_test)
torch.save(X_tfidfvect_test_tensor, 'X_tfidfvect_test_tensor_4_days_test.pt')
torch.save(y_tfidfvect_test_tensor, 'y_tfidfvect_test_tensor_4_days_test.pt')
# torch.save(X_static_test_tensor, 'X_static_test_tensor_4_days_test.pt')
# torch.save(X_dynamic_test_tensor, 'X_dynamic_test_tensor_4_days_test.pt')
torch.save(X_structured_test_tensor_tfidf, 'X_structured_test_tensor_tfidf_4_days_subset.pt')

In [13]:
#print(X_static_train_tensor.size())
#print(X_dynamic_train_tensor.size())
print(X_structured_train_tensor_tfidf.size())
print(X_tfidfvect_train_tensor.size())

print(X_structured_train_tensor_tfidf[0])
print(X_structured_train_tensor_tfidf[1])

torch.Size([4673, 4])
torch.Size([4673, 32, 200])
tensor([65.,  0., 26., 21.])
tensor([65.,  0., 26., 21.])


In [14]:
# from create_tensors import max_notes, tensors
# import torch

# '''REMEMBER: possible to manually set look_back and look_ahead
# Look_back must be set to the same in both max_notes and tensors to create correct padding.
# (default look_back = 4, look_ahead = 3)'''

# maximum_notes_train = max_notes(df_sentence_train, df_admissions_train)
# print(maximum_notes_train)

# maximum_notes_test = max_notes(df_sentence_test, df_admissions_test)
# print(maximum_notes_test)

# maximum_notes = max(maximum_notes_train, maximum_notes_test)
# print(maximum_notes)

# X_sentence_train_tensor, y_sentence_train_tensor, X_structured_train_tensor_sentence = tensors(df_sentence_train, y_train_df, df_admissions_train, maximum_notes, predictor_dict_train)
# torch.save(X_sentence_train_tensor, 'X_sentence_train_tensor_4_days.pt')
# torch.save(y_sentence_train_tensor, 'y_sentence_train_tensor_4_days.pt')
# torch.save(X_structured_train_tensor_sentence, 'X_structured_train_tensor_sentence_4_days.pt')

# X_sentence_test_tensor, y_sentence_test_tensor, X_structured_test_tensor_sentence = tensors(df_sentence_test, y_test_df, df_admissions_test, maximum_notes, predictor_dict_test)
# torch.save(X_sentence_test_tensor, 'X_sentence_test_tensor_4_days.pt')
# torch.save(y_sentence_test_tensor, 'y_sentence_test_tensor_4_days.pt')
# torch.save(X_structured_test_tensor_sentence, 'X_structured_test_tensor_sentence_4_days.pt')


In [15]:
# from create_tensors import max_notes, tensors
# import torch

# '''REMEMBER: possible to manually set look_back and look_ahead
# Look_back must be set to the same in both max_notes and tensors to create correct padding.
# (default look_back = 4, look_ahead = 3)'''


# #hvad hvis der er en i testsættet der har flere notes?? skal vel tjekke begge og tage den med flest??
# maximum_notes_train = max_notes(df_tfidfvect_train, df_admissions_train)
# print(maximum_notes_train)

# maximum_notes_test = max_notes(df_tfidfvect_test, df_admissions_test)
# print(maximum_notes_test)

# maximum_notes = max(maximum_notes_train, maximum_notes_test)
# print(maximum_notes)

# #train
# X_tfidfvect_train_tensor, y_tfidfvect_train_tensor, X_structured_train_tensor_tfidf = tensors(df_tfidfvect_train, y_train_df, df_admissions_train, maximum_notes, predictor_dict_train)
# torch.save(X_tfidfvect_train_tensor, 'X_tfidfvect_train_tensor_4_days.pt')
# torch.save(y_tfidfvect_train_tensor, 'y_tfidfvect_train_tensor_4_days.pt')
# torch.save(X_structured_train_tensor_tfidf, 'X_structured_train_tensor_tfidf_4_days.pt')

# #test
# X_tfidfvect_test_tensor, y_tfidfvect_test_tensor, X_structured_test_tensor_tfidf = tensors(df_tfidfvect_test, y_test_df, df_admissions_test, maximum_notes, predictor_dict_test)
# torch.save(X_tfidfvect_test_tensor, 'X_tfidfvect_test_tensor_4_days.pt')
# torch.save(y_tfidfvect_test_tensor, 'y_tfidfvect_test_tensor_4_days.pt')
# torch.save(X_structured_test_tensor_tfidf, 'X_structured_test_tensor_tfidf_4_days.pt')


In [16]:
######################## KØR HERTIL! ###############################

### LOADING TENSORS 

In [17]:
import torch 
import numpy as np

# '''Sentence'''
# #train
# X_sentence_train_tensor = torch.load('X_sentence_train_tensor_4_days.pt', map_location=torch.device('cuda'))
# y_sentence_train_tensor = torch.load('y_sentence_train_tensor_4_days.pt', map_location=torch.device('cuda'))

# #test
# X_sentence_test_tensor = torch.load('X_sentence_test_tensor_4_days.pt', map_location=torch.device('cuda'))
# y_sentence_test_tensor = torch.load('y_sentence_test_tensor_4_days.pt', map_location=torch.device('cuda'))

# print(X_sentence_train_tensor.size())
# print(y_sentence_train_tensor.size())
# print(X_sentence_test_tensor.size())
# print(y_sentence_test_tensor.size())



# # '''Tf-idf'''
# # #train
# X_tfidfvect_train_tensor = torch.load('X_tfidfvect_train_tensor_4_days.pt', map_location=torch.device('cuda'))
# y_tfidfvect_train_tensor = torch.load('y_stfidfvect_train_tensor_4_days.pt', map_location=torch.device('cuda'))

# #test
# X_tfidfvect_test_tensor = torch.load('X_tfidfvect_test_tensor_4_days.pt', map_location=torch.device('cuda'))
# y_tfidfvect_test_tensor = torch.load('y_tfidfvect_test_tensor_4_days.pt', map_location=torch.device('cuda'))

# print(X_tfidfvect_train_tensor.size())
# print(y_tfidfvect_train_tensor.size())
# print(X_tfidfvect_test_tensor.size())
# print(y_tfidfvect_test_tensor.size())



In [18]:
'''Tf-idf SUBSETS'''
#train
X_tfidfvect_train_tensor = torch.load('X_tfidfvect_train_tensor_4_days_subset.pt', map_location=torch.device('cuda'))
y_tfidfvect_train_tensor = torch.load('y_tfidfvect_train_tensor_4_days_subset.pt', map_location=torch.device('cuda'))
X_structured_train_tensor_tfidf = torch.load('X_structured_train_tensor_tfidf_4_days_subset.pt', map_location=torch.device('cuda'))

#test
X_tfidfvect_test_tensor = torch.load('X_tfidfvect_test_tensor_4_days_subset.pt', map_location=torch.device('cuda'))
y_tfidfvect_test_tensor = torch.load('y_tfidfvect_test_tensor_4_days_subset.pt', map_location=torch.device('cuda'))
X_structured_test_tensor_tfidf = torch.load('X_structured_test_tensor_tfidf_4_days_subset.pt', map_location=torch.device('cuda'))


print(X_tfidfvect_train_tensor.size())
print(y_tfidfvect_train_tensor.size())
print(X_structured_train_tensor_tfidf.size())


print(X_tfidfvect_test_tensor.size())
print(y_tfidfvect_test_tensor.size())
print(X_structured_test_tensor_tfidf.size())


torch.Size([4673, 32, 200])
torch.Size([4673])
torch.Size([4673, 4])
torch.Size([1893, 32, 200])
torch.Size([1893])
torch.Size([1893, 4])


In [19]:
# #Gets them to same dimension, but works worse???
y_tfidfvect_train_tensor = torch.unsqueeze(y_tfidfvect_train_tensor, 1).cuda()
y_tfidfvect_test_tensor = torch.unsqueeze(y_tfidfvect_test_tensor, 1).cuda()

y_tfidfvect_test_tensor.size()

torch.Size([1893, 1])

In [21]:
print(y_tfidfvect_train_tensor.device)
print(y_tfidfvect_test_tensor.device)
print(X_tfidfvect_train_tensor.device)
print(X_tfidfvect_test_tensor.device)
print(X_structured_train_tensor_tfidf.device)
print(X_structured_test_tensor_tfidf.device)

cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


In [22]:
num_epochs = 10 #10 epochs
learning_rate = 0.001 #0.001 lr

input_size = 200 #number of features
hidden_size = 60 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes 
seq_length = X_tfidfvect_train_tensor.shape[1]

## LSTM 

In [7]:
import torch 



In [23]:
print(torch.cuda.is_available())

#Device ID of the GPU
print(torch.cuda.current_device())

#Device name of the GPU
print(torch.cuda.get_device_name(0))

#memory used on GPU
print(torch.cuda.memory_allocated())

True
0
NVIDIA A100-PCIE-40GB
168222208


In [25]:
# #Testing
# from LSTM import LSTM1
# my_LSTM = LSTM1(num_classes, input_size, hidden_size, num_layers, seq_length)

# cuda_LSTM = my_LSTM.to("cuda")

# for param in cuda_LSTM.parameters():
#     print (param.device)

cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


In [27]:
from LSTM import LSTM1
import torch #pytorch
import math
from batches import batch

#instantiating LSTM1 object
my_LSTM = LSTM1(num_classes, input_size, hidden_size, num_layers, seq_length).to("cuda")

for param in my_LSTM.parameters():
    print (param.device)


criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(my_LSTM.parameters(), lr=learning_rate) 

# print(list(batches_x)[0])
# n_batches = sum(1 for _ in batches_x)
# print(n_batches)
# print(list(batches_x)[0])


#training loop
for epoch in range(num_epochs):
    
    batch_size = 32
    batches_x = batch(X_tfidfvect_train_tensor, batch_size)
    batches_y = batch(y_tfidfvect_train_tensor, batch_size)

    n_batches = math.ceil(len(X_tfidfvect_train_tensor)/batch_size)
    
    for i in range(n_batches):
        
        batch_x = next(batches_x)
        for j in batch_x:
            batch_x_new = torch.stack(batch_x)
        
        batch_y = next(batches_y)
        for k in batch_y:
            batch_y_new = torch.stack(batch_y)

        outputs = my_LSTM.forward(batch_x_new)# (X_train_tensor) #forward pass
        optimizer.zero_grad() #calculate the gradient, manually setting to 0

        #obtaining loss function
        loss = criterion(outputs, batch_y_new) 

        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
    if epoch % 2 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 




cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
Epoch: 0, loss: 0.00015
Epoch: 2, loss: 0.00019
Epoch: 4, loss: 0.00016
Epoch: 6, loss: 0.00015
Epoch: 8, loss: 0.00014


In [28]:
batches_x = batch(X_tfidfvect_test_tensor, batch_size)
batches_y = batch(y_tfidfvect_test_tensor, batch_size)

n_batches = math.ceil(len(X_tfidfvect_test_tensor)/batch_size)
    
test_loss = 0.0
# Generate predictions for the test dataset
predictions = []

with torch.no_grad():
    for i in range(n_batches):
        
        batch_x = next(batches_x)
        for j in batch_x:
            batch_x_new = torch.stack(batch_x)
        
        batch_y = next(batches_y)
        for k in batch_y:
            batch_y_new = torch.stack(batch_y)


        # Forward pass
        outputs = my_LSTM(batch_x_new)

        loss = criterion(outputs, batch_y_new)
        test_loss += loss.item()

        # Save the predictions
        predictions += outputs.squeeze().tolist()
        

    # Compute the evaluation metrics
    avg_test_loss = test_loss / len(X_tfidfvect_test_tensor)
    print('Average Test Loss: {:.4f}'.format(avg_test_loss))


# Convert the predictions and actual values to numpy arrays
predictions = np.array(predictions)
actuals = y_tfidfvect_test_tensor.cpu().numpy()


Average Test Loss: 0.0007


In [29]:
len(predictions)

len(y_tfidfvect_test_tensor)

1893

In [30]:
from sklearn.metrics import classification_report
#[print(i) for i in predictions]

#HVORFOR GÆTTER DEN KUN 1???? 
print(max(np.unique(predictions)))

# Applying transformation to get binary values predictions with 0.5 as thresold
binary_predictions = list(map(lambda x: 0 if x < 0.5 else 1, predictions))


print(classification_report(y_tfidfvect_test_tensor.cpu(), binary_predictions))

0.011317039839923382
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      1866
         1.0       0.00      0.00      0.00        27

    accuracy                           0.99      1893
   macro avg       0.49      0.50      0.50      1893
weighted avg       0.97      0.99      0.98      1893



E:\ProgramData\Anaconda3\envs\linapd\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\envs\linapd\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\ProgramData\Anaconda3\envs\linapd\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(f"True Positives: {TP}. False Positives: {FP}. True negatives: {TN}. False negatives: {FN}.")

In [32]:
print(perf_measure(actuals, binary_predictions))


True Positives: 0. False Positives: 0. True negatives: 1866. False negatives: 27.
